# Aufgaben Blatt 4 KI Machine Learning I



## Aufgabe 1 (Lineare Regression)

Bearbeiten Sie die Aufgabe https://github.com/oduerr/ki/blob/main/linear_regression/lr_gradient_descent.ipynb

Versuchen Sie den Code zu verstehen und machen die kleineren Aufgaben, die in dem notebook besprochen werden.

## Aufgabe 2 (Titanic)
In dieser Aufgabe nehmen Sie an der Titanic Challenge (https://www.kaggle.com/c/titanic) teil. Sie können die Aufgabe am eigenen PC lösen oder direkt in Kaggle lösen. Die Daten liegen auch auf Moodle. 

a) Lesen Sie die Trainingsdaten ein und teilen Sie sie in ein Validierungsdatenset (20%) und in ein eigentliches Trainigsdatenset (80%) auf. Finden Sie auf dem Trainigsdatenset eine Regel für das Überleben alleine aufgrund der Klasse des Tickets (Pclass). Wenden Sie diese Regel auf die Validierungsdaten an. Wie gut ist die Genauigkeit (Anteil der korrekten Klassifikationen) auf den Validierungsdaten?  

In [2]:
# Hinweise zum Einlesen
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
train_val = pd.read_csv('titanic/train.csv')

# Hinweise zum Erzeugen einer Tabelle
# pd.crosstab(...)

# Hinweise um die Accuracy zu berechnen
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.model_selection import train_test_split

training_set, validation_set = train_test_split(train_val, test_size=0.2)
print(len(training_set))
print(len(validation_set))

## wer hat überlebt

classes = {
    1: (0, 0),
    2: (0, 0),
    3: (0, 0)
}

for person in training_set.itertuples(index=False):
    key = person[2]
    current_tuple = classes[key]
    updated_tuple = (current_tuple[0] + 1, current_tuple[1] + person[1])
    classes[key] = updated_tuple

for key, value in classes.items():
    total = value[0]
    survived = value[1]
    ratio = survived / total if total != 0 else 0
    print(f"Klasse {key}: Überlebende/Tot = {survived}/{total} Verhältnis = {ratio}")

class_survival_rate = {}

for key, value in classes.items():
    total = value[0]
    survived = value[1]
    ratio = survived / total if total != 0 else 0
    class_survival_rate[key] = ratio

print(class_survival_rate)

## Vorhersage

import random
val_predictions = []
for passenger in validation_set.itertuples(index=False):
    predicted_survival = 1 if random.random() < class_survival_rate[passenger[2]] else 0
    val_predictions.append(predicted_survival)

correct_predictions = validation_set['Survived'] == val_predictions
print(f"Vorhersagen: {sum(correct_predictions)}")
accuracy = sum(correct_predictions) / len(correct_predictions)
print("Genauigkeit:", accuracy)


from pathlib import Path

original_test_set = pd.read_csv("titanic/test.csv")

val_predictions = []
for passenger in original_test_set.itertuples():
    predicted_survival = 1 if random.random() < class_survival_rate[passenger[2]] else 0
    val_predictions.append(predicted_survival)

output = pd.DataFrame({'PassengerId': original_test_set["PassengerId"], 'Survived': val_predictions})
output.to_csv('my_submission.csv', index=False)


712
179
Klasse 1: Überlebende/Tot = 109/170 Verhältnis = 0.6411764705882353
Klasse 2: Überlebende/Tot = 68/154 Verhältnis = 0.44155844155844154
Klasse 3: Überlebende/Tot = 91/388 Verhältnis = 0.2345360824742268
{1: 0.6411764705882353, 2: 0.44155844155844154, 3: 0.2345360824742268}
Vorhersagen: 108
Genauigkeit: 0.6033519553072626


b) Wenden Sie die Regel aus a) auf die Testdaten an und laden Sie Ihre Lösung hoch. 

In [4]:
# 0.56689 war das Ergebnis

c) Logistische Regression mit Pclass

Trainieren Sie eine logistische Regression mit den Variablen 'Pclass'. Verwenden Sie die Klasse `sklearn.linear_model.LogisticRegression`. Berechnen Sie die Accuracy auf dem Validierungsset.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logReg = LogisticRegression()
logReg.fit(training_set[["Pclass"]], training_set[["Survived"]])

prediction = logReg.predict(validation_set[["Pclass"]])

accuracy = accuracy_score(validation_set[["Survived"]], prediction)
print("Genauigkeit:", accuracy)

print(logReg.score(validation_set[["Pclass"]], validation_set[["Survived"]]))

Genauigkeit: 0.6312849162011173
0.6312849162011173


/Users/felixfischer/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


d) Coding / Feature engineering 

d.i) Missing Values:

Verwenden Sie nun weitere Features. Die Variable Age enthält Missing values, die Sie durch folgenden code ersetzen können (was passiert da?)

In [25]:
val["Age"].fillna(train["Age"].median(skipna=True), inplace=True)
train["Age"].fillna(train["Age"].median(skipna=True), inplace=True)

/var/folders/lr/n_w7l2nn1klg5pbt8vrzdt7m0000gn/T/ipykernel_87315/765635614.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["Age"].fillna(train_data["Age"].median(skipna=True), inplace=True)
/var/folders/lr/n_w7l2nn1klg5pbt8vrzdt7m0000gn/T/ipykernel_87315/765635614.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which

d.ii) Kategorische Variable

Verwenden Sie die Funktion `pd.get_dummies` um die Variablen 'Pclass' and 'Sex' in numerische Werte umzuwandeln. Führen Sie nun eine logistische Regression durch.

e) Weitere Klassifikatoren. Neben der logistischen Regression, gibt es weitere Klassifikatoren. Der Random-Forest ist ein recht stabiler Klassifikator, was wäre die Performance von diesem Klassifikator.

In [7]:
# Hinweise zur Lösung
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
#rf hat nun gleiches interface, wie die logistische Regression

f) [optional] Versuchen Sie weitere Features zu erzeugen und laden den besten Klassifikator auf Kaggle hoch. 

## Aufgabe 3 Titanic mit Neuronalen Netzen 

Hinweis: Diese Aufgabe kann erst nach der dritten Vorlesung in ML gemacht werden.

Mit den gleichen Daten, wie in der Aufgabe 2 d. Erstellen Sie ein fully connected neural network und fitten es an die Ttrainingsdaten. Verwenden Sie mindestens zwei hidden Layer. Plotten Sie den Verlauf der Loss Kurve für die Trainings- und Validierungsdaten. Optional: Laden Sie Ihre beste Lösung auf Kaggle hoch. 

In [19]:
#Sie können von folgendem Code starten um das Netzwerk zu definieren, füllen Sie die ...
import tensorflow as tf
import tensorflow.keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(..., activation='sigmoid', batch_input_shape=(None, 4))) #We have 4 input features
#...
model.add(Dense(1, activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

ValueError: Unrecognized keyword arguments passed to Dense: {'batch_input_shape': (None, 4)}